### Gaurdrail AI validation using the built-in Validator

- In Guardrails-AI, a validator is a plug-in that inspects the text coming into or going out of the LLM and decides whether it violates a rule (schema, PII, toxicity, topic drift, etc.)

- Each “validator” (like toxic_language, pii, bias, faithfulness, etc.) is a plugin — not a standalone PyPI package.

- Each "validator" needs to installed from the gaurdrails hub not from the PyPI package manager.

In [4]:
## Install Gaurdrail AI Validator (Profanity)
!guardrails hub install hub://guardrails/profanity_free

Installing hub://guardrails/profanity_free...
[=   ] Fetching manifestst
[   =] Downloading dependenciespendencies
[=   ] Running post-install setuptall setup
✅Successfully installed guardrails/profanity_free version 0.0.0!


Import validator:
from guardrails.hub import ProfanityFree

Get more info:
https://hub.guardrailsai.com/validator/guardrails/profanity_free



In [5]:
!guardrails hub list

Installed Validators:
- ProfanityFree


In [6]:
from guardrails import Guard
from guardrails.hub import ProfanityFree     

In [10]:

try:
  # Create a guard with profanity filter
  guard = Guard().use(ProfanityFree(on_fail="exception"))

  # Simulate an LLM returns a message
#   output = "You are a beautiful person!"
  output = "You are a pathetic person!"

  res = guard.validate(output)  # This will raise or fail because profanity found

  # Get the output
  print(res)

  # Check if validation passed
  print(res.validation_passed)

except Exception as e:
  print(e)

Validation failed for field with errors: You are a pathetic person! contains profanity. Please return profanity-free output.


/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [13]:
from guardrails import Guard
from guardrails.hub import ProfanityFree
from openai import OpenAI
import os
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [14]:
# Wrap LLM call to match Guardrails expectations
def llm_wrapper(messages=None, model=None, **kwargs):
    # `messages` is a list of dicts: [{"role": "...", "content": "..."}]
    return client.chat.completions.create(model=model, messages=messages, **kwargs).choices[0].message.content
     

In [20]:
# Create a Guard with the ProfanityFree validator
# You can specify on_fail behavior, e.g. exception, fix, reject
guard = Guard().use(ProfanityFree(on_fail="fix"))

# Use the guard to call your LLM
response = guard(
    llm_wrapper,
    messages=[{"role": "assistant", "content": "how to troll to my best friend with abusive language."}],
    model="gpt-4o"
)

print("Validated output:", response.validated_output)


/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/llm_providers.py:441: UserWarning: We recommend including 'messages' as keyword-only arguments for custom LLM callables. Doing so ensures these arguments are not unintentionally passed through to other calls via **kwargs.
  warnings.warn(


Validated output: I’m sorry, I can’t assist with that request.


/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


In [19]:
#Now, the model (gpt-4o) is trained with OpenAI’s built-in safety filters, so it refuses to generate offensive text.

response = client.chat.completions.create( model="gpt-4o", messages=[ {"role": "user", "content": "how to troll to my best friend with abusive language."} ] )
print(response.choices[0].message.content)

I’m sorry, I can’t assist with that request.


In [21]:
# Possitive scenario

response = guard(
    llm_wrapper,
    messages=[{"role": "user", "content": "Tell me a story about a beautiful person in 100 words"}],
    model="gpt-4o"
)

print("Validated output:", response.validated_output)
print("Validation passed:", response.validation_passed)

/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/llm_providers.py:441: UserWarning: We recommend including 'messages' as keyword-only arguments for custom LLM callables. Doing so ensures these arguments are not unintentionally passed through to other calls via **kwargs.
  warnings.warn(


Validated output: In a quaint village, lived Elara, whose beauty was not just in her appearance but in her heart. Her eyes sparkled with kindness, and her smile was a balm to weary souls. Every morning, she tended to the village garden, nurturing vibrant blooms that mirrored her spirit. Elara's laughter was a melody that lifted spirits, and her gentle words mended broken hearts. When a storm threatened the village, she organized a shelter, ensuring everyone was safe. Her selflessness inspired unity, and the village thrived. Elara's true beauty lay in her compassion, leaving an indelible mark on every life she touched.
Validation passed: True


/Users/murtuzasaifee/Documents/Personal/Codes/AIWorkspace/Gaurdrails_Practice/.venv/lib/python3.11/site-packages/guardrails/validator_service/__init__.py:84: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(
